In [14]:
import sys

sys.path.append("../")

In [15]:
import pandas as pd
import numpy as np
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from stats_arb.tests import adf_test, kpss_test, cal_half_life, pp_test
from datetime import datetime, timedelta

import matplotlib.pyplot as plt

# DATA_PATH = '/mnt/d/Trading/trading-agent/crypto-pair-trading/data/5min'
# DATA_PATH = '/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/crypto/1h'
DATA_PATH = '/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/us_stocks/stocks'

In [16]:
table_S = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
table_S = table_S[0]
symbols = table_S.Symbol.tolist()

In [20]:
selected_symbols = ['BDX', 'BIIB']

start_date = '2019-01-01'
end_date = '2022-11-30'
dfs = {}
data = []

for symbol in selected_symbols:
    try:
        file = f'{DATA_PATH}/{symbol}.csv'
        # print(file)
        df = pd.read_csv(file, parse_dates=['Date'])
        df.set_index('Date', inplace=True)
        df.columns = ['open', 'high', 'low', 'close', 'adj_close', 'volume']
        df = df[(df.index > start_date) & (df.index < end_date)]
        df = df[~df.index.duplicated(keep='first')]

        dfs[symbol] = df
        
        # print(symbol, df.index[-1])
        df.rename(columns={'adj_close': symbol}, inplace=True)
        # the data is too long, just limit to recent period
        data.append(np.log(df[symbol]))
    except FileNotFoundError:
        # print(symbol, 'not found')
        pass

df = pd.concat(data, axis=1)
# df = df.dropna(axis=1, how='all')
df.dropna(inplace=True, how='any')

df.tail()

,BDX,BIIB
Date,,
2022-11-09,5.385824,5.643006
2022-11-10,5.434944,5.672292
2022-11-11,5.427897,5.667983
2022-11-14,5.412360,5.700644
2022-11-15,5.404343,5.703049


In [21]:
p = 1
COINTEGRATION_CONFIDENCE_LEVEL = 90

# the 90%, 95%, and 99% confidence levels for the trace statistic and maximum 
# eigenvalue statistic are stored in the first, second, and third column of 
# cvt and cvm, respectively
confidence_level_cols = {
    90: 0,
    95: 1,
    99: 2
}
confidence_level_col = confidence_level_cols[COINTEGRATION_CONFIDENCE_LEVEL]

def test_johansen(df, symbol_pairs):
    df_t = df[symbol_pairs].copy()
    df_t.dropna(axis=1, how='all', inplace=True)

    # The second and third parameters indicate constant term, with a lag of 1. 
    result = coint_johansen(df_t, 0, p)

    trace_crit_value = result.cvt[:, confidence_level_col]
    eigen_crit_value = result.cvm[:, confidence_level_col]
    # print("trace_crit_value",trace_crit_value)
    # print("eigen_crit_value",eigen_crit_value)
    # print("lr1",result.lr1)
    # print("lr2",result.lr2)

    # The trace statistic and maximum eigenvalue statistic are stored in lr1 and lr2;
    # see if they exceeded the confidence threshold
    if np.all(result.lr1 >= trace_crit_value) and np.all(result.lr2 >= eigen_crit_value):
        # print(f"{symbol_pairs} are cointegrated")
        # The first i.e. leftmost column of eigenvectors matrix, result.evec, contains the best weights.
        v1= result.evec[:,0:1]
        hr=v1/-v1[1] #to get the hedge ratio divide the best_eigenvector by the negative of the second component of best_eigenvector
        #the regression will be: close of symbList[1] = hr[0]*close of symbList[0] + error
        #where the beta of the regression is hr[0], also known as the hedge ratio, and
        #the error of the regression is the mean reverting residual signal that you need to predict, it is also known as the "spread"
        #the spread = close of symbList[1] - hr[0]*close of symbList[0] or alternatively (the same thing):
        #do a regression with close of symbList[0] as x and lose of symbList[1] as y, and take the residuals of the regression to be the spread.
        coint_pair = dict(hedge_ratio=v1[:, 0])
        for i, s in enumerate(symbol_pairs):
            coint_pair[f'sid_{i+1}'] = s

        return coint_pair

test_johansen(df, selected_symbols)

{'hedge_ratio': array([18.47518862,  2.51962775]),
 'sid_1': 'BDX',
 'sid_2': 'BIIB'}